## POS tagging using modified Viterbi

### Data Preparation

In [1]:
#Importing libraries
import nltk
from sklearn.model_selection import train_test_split
import random
import numpy as np
import pandas as pd

In [2]:
# reading the Treebank tagged sentences
nltk_data = list(nltk.corpus.treebank.tagged_sents(tagset='universal'))

In [3]:
print(nltk_data[:3])

[[('Pierre', 'NOUN'), ('Vinken', 'NOUN'), (',', '.'), ('61', 'NUM'), ('years', 'NOUN'), ('old', 'ADJ'), (',', '.'), ('will', 'VERB'), ('join', 'VERB'), ('the', 'DET'), ('board', 'NOUN'), ('as', 'ADP'), ('a', 'DET'), ('nonexecutive', 'ADJ'), ('director', 'NOUN'), ('Nov.', 'NOUN'), ('29', 'NUM'), ('.', '.')], [('Mr.', 'NOUN'), ('Vinken', 'NOUN'), ('is', 'VERB'), ('chairman', 'NOUN'), ('of', 'ADP'), ('Elsevier', 'NOUN'), ('N.V.', 'NOUN'), (',', '.'), ('the', 'DET'), ('Dutch', 'NOUN'), ('publishing', 'VERB'), ('group', 'NOUN'), ('.', '.')], [('Rudolph', 'NOUN'), ('Agnew', 'NOUN'), (',', '.'), ('55', 'NUM'), ('years', 'NOUN'), ('old', 'ADJ'), ('and', 'CONJ'), ('former', 'ADJ'), ('chairman', 'NOUN'), ('of', 'ADP'), ('Consolidated', 'NOUN'), ('Gold', 'NOUN'), ('Fields', 'NOUN'), ('PLC', 'NOUN'), (',', '.'), ('was', 'VERB'), ('named', 'VERB'), ('*-1', 'X'), ('a', 'DET'), ('nonexecutive', 'ADJ'), ('director', 'NOUN'), ('of', 'ADP'), ('this', 'DET'), ('British', 'ADJ'), ('industrial', 'ADJ'), ('

In [4]:
# splitting into train and test
random.seed(1234)
train_set, test_set = train_test_split(nltk_data, test_size=0.05,random_state=42)

print(len(train_set))
print(len(test_set))
print(train_set[:2])

3718
196
[[('Bank', 'NOUN'), ('of', 'ADP'), ('New', 'NOUN'), ('England', 'NOUN'), ("'s", 'PRT'), ('shares', 'NOUN'), ('are', 'VERB'), ('traded', 'VERB'), ('*-1', 'X'), ('on', 'ADP'), ('the', 'DET'), ('New', 'NOUN'), ('York', 'NOUN'), ('Stock', 'NOUN'), ('Exchange', 'NOUN'), ('.', '.')], [('$', '.'), ('130', 'NUM'), ('million', 'NUM'), ('*U*', 'X'), ('of', 'ADP'), ('general', 'ADJ'), ('obligation', 'NOUN'), ('distributable', 'ADJ'), ('state', 'NOUN'), ('aid', 'NOUN'), ('bonds', 'NOUN'), ('due', 'ADJ'), ('1991-2000', 'NUM'), ('and', 'CONJ'), ('2009', 'NUM'), (',', '.'), ('tentatively', 'ADV'), ('priced', 'VERB'), ('*', 'X'), ('by', 'ADP'), ('a', 'DET'), ('Chemical', 'NOUN'), ('Securities', 'NOUN'), ('Inc.', 'NOUN'), ('group', 'NOUN'), ('*', 'X'), ('to', 'PRT'), ('yield', 'VERB'), ('from', 'ADP'), ('6.20', 'NUM'), ('%', 'NOUN'), ('in', 'ADP'), ('1991', 'NUM'), ('to', 'PRT'), ('7.272', 'NUM'), ('%', 'NOUN'), ('in', 'ADP'), ('2009', 'NUM'), ('.', '.')]]


### Build the vanilla Viterbi based POS tagger

In [5]:
# Getting list of tagged words
train_tagged_words = [tup for sent in train_set for tup in sent]
print('Total Tagges Words',len(train_tagged_words))

# tokens 
tokens = [pair[0] for pair in train_tagged_words]

#Check some sentence data
tokens[:10]

Total Tagges Words 95589


['Bank', 'of', 'New', 'England', "'s", 'shares', 'are', 'traded', '*-1', 'on']

#### Interface for converting POS tags from various treebanks to the universal tagset of Petrov, Das, & McDonald.

The tagset consists of the following 12 coarse tags:
    
    VERB - verbs (all tenses and modes)
    NOUN - nouns (common and proper)
    PRON - pronouns
    ADJ - adjectives
    ADV - adverbs
    ADP - adpositions (prepositions and postpositions)
    CONJ - conjunctions
    DET - determiners
    NUM - cardinal numbers
    PRT - particles or other function words
    X - other: foreign words, typos, abbreviations
    . - punctuation

In [6]:
# vocabulary
V = set(tokens)
print(len(V))

# number of tags
T = set([pair[1] for pair in train_tagged_words])
len(T)

12109


12

In [7]:
# Emission Probabilities
# computing P(w/t) and storing in T x V matrix
t = len(T)
v = len(V)
w_given_t = np.zeros((t, v))

In [8]:
# Check the train set shape
w_given_t.shape

(12, 12109)

### Emission Probabilities

In [9]:
# compute word given tag: Emission Probability
from collections import Counter
tags = [pair[1] for pair in train_tagged_words]
tag_counts = Counter(tags)

all_Tags = list(set([pair[1] for pair in train_tagged_words]))
tag_word_dict = dict()
for t in all_Tags:
    tag_word_dict.update({t:[pair[0] for pair in train_tagged_words if pair[1]== t]})

def word_given_tag(word, tag, train_bag = train_tagged_words):
    count_tag = tag_counts.get(tag)
    tag_list = tag_word_dict.get(tag)
    count_w_given_tag = tag_list.count(word)
    return (count_w_given_tag, count_tag)

### Transition Probabilities

In [10]:
# compute tag given tag: tag2(t2) given tag1 (t1), i.e. Transition Probability
def t2_given_t1(t2, t1, train_bag = train_tagged_words):
    tags = [pair[1] for pair in train_bag]
    count_t1 = len([t for t in tags if t==t1])
    count_t2_t1 = 0
    for index in range(len(tags)-1):
        if tags[index]==t1 and tags[index+1] == t2:
            count_t2_t1 += 1
    return (count_t2_t1, count_t1)

In [11]:
# creating t x t transition matrix of tags
# each column is t2, each row is t1
# thus M(i, j) represents P(tj given ti)

tags_matrix = np.zeros((len(T), len(T)), dtype='float32')
for i, t1 in enumerate(list(T)):
    for j, t2 in enumerate(list(T)): 
        tags_matrix[i, j] = t2_given_t1(t2, t1)[0]/t2_given_t1(t2, t1)[1]

In [12]:
# convert the matrix to a df for better readability
tags_df = pd.DataFrame(tags_matrix, columns = list(T), index=list(T))

# check the shape
print('Shape:',tags_df.shape)

print('Transition probs:')
tags_df

Shape: (12, 12)
Transition probs:


,VERB,PRT,ADV,NOUN,X,DET,.,ADJ,PRON,NUM,ADP,CONJ
VERB,0.169189,0.031121,0.082577,0.110904,0.218005,0.133101,0.035312,0.064649,0.036244,0.022817,0.090493,0.005588
PRT,0.402746,0.001635,0.010134,0.242563,0.013403,0.100360,0.041517,0.086303,0.018960,0.058516,0.021576,0.002288
ADV,0.344518,0.013621,0.081063,0.030897,0.023588,0.067110,0.136877,0.130233,0.015615,0.030565,0.119601,0.006312
NOUN,0.147978,0.043832,0.016884,0.264632,0.029136,0.013310,0.239179,0.012289,0.004923,0.009627,0.176275,0.041936
X,0.203633,0.185787,0.025175,0.061345,0.076482,0.055131,0.163799,0.016571,0.056087,0.002709,0.142925,0.010357
DET,0.038387,0.000241,0.012313,0.640029,0.045509,0.005311,0.017986,0.204973,0.003742,0.021970,0.009054,0.000483
.,0.088505,0.002339,0.052078,0.223152,0.026623,0.173502,0.093812,0.044972,0.065389,0.080500,0.091114,0.057924
ADJ,0.012342,0.010861,0.004608,0.696725,0.021392,0.005101,0.065328,0.066645,0.000658,0.020405,0.078986,0.016949
PRON,0.480901,0.012223,0.033995,0.211230,0.092819,0.009549,0.041253,0.074866,0.008021,0.007257,0.022918,0.004966
NUM,0.017544,0.027951,0.002974,0.355338,0.206661,0.003271,0.118347,0.034196,0.001487,0.184062,0.034790,0.013381


In [13]:
# Viterbi Heuristic
def Viterbi(words, train_bag = train_tagged_words):
    state = []
    T = list(set([pair[1] for pair in train_bag]))
    
    for key, word in enumerate(words):
        #initialise list of probability column for a given observation
        p = [] 
        for tag in T:
            if key == 0:
                transition_p = tags_df.loc['.', tag]
            else:
                transition_p = tags_df.loc[state[-1], tag]
                
            # compute emission and state probabilities
            emission_out = word_given_tag(words[key], tag)
            emission_p = emission_out[0]/emission_out[1]
            state_probability = emission_p * transition_p    
            p.append(state_probability)
            
        pmax = max(p)
        # getting state for which probability is maximum
        state_max = T[p.index(pmax)] 
        state.append(state_max)
    return list(zip(words, state))

In [14]:
# list of untagged words
test_tagged_words = [tup[0] for sent in test_set for tup in sent]

In [15]:
len(test_tagged_words)

5087

In [16]:
import time
# tagging the test sentences
start = time.time()
tagged_seq = Viterbi(test_tagged_words)
end = time.time()
difference = end-start

In [17]:
print("Time taken in seconds: ", difference)

Time taken in seconds:  43.12033820152283


In [18]:
print('Tagged test words')
tagged_seq[:5]

Tagged test words


[('For', 'ADP'),
 ('the', 'DET'),
 ('Agency', 'NOUN'),
 ('for', 'ADP'),
 ('International', 'NOUN')]

In [19]:
print("Time taken in seconds: ", difference)
test_taggings = [tup for sent in test_set for tup in sent]
wronglytagged_words =[]
for i in range(len(tagged_seq)):
    if test_taggings[i][1] != tagged_seq[i][1]:
        wronglytagged_words.append((tagged_seq[i],test_taggings[i]))
print('Total wrongly tagged words',len(wronglytagged_words))
distinct_words = list(set([pair[0] for pair in wronglytagged_words]))

print('Distinct Word Count',len(distinct_words))

# accuracy
check = [i for i, j in zip(tagged_seq, test_taggings) if i == j] 
accuracy = len(check)/len(tagged_seq)
print(accuracy)

Time taken in seconds:  43.12033820152283
Total wrongly tagged words 385
Distinct Word Count 347
0.92431688618046


In [20]:
wronglytagged_words[:5]

[(('trade', 'VERB'), ('trade', 'NOUN')),
 (('Overseas', 'VERB'), ('Overseas', 'NOUN')),
 (('Private', 'ADJ'), ('Private', 'NOUN')),
 (('pre-1917', 'VERB'), ('pre-1917', 'ADJ')),
 (('Unemployment', 'VERB'), ('Unemployment', 'NOUN'))]

### Solve the problem of unknown words

    VERB - verbs (all tenses and modes)
    NOUN - nouns (common and proper)
    PRON - pronouns
    ADJ - adjectives
    ADV - adverbs
    ADP - adpositions (prepositions and postpositions)
    CONJ - conjunctions
    DET - determiners
    NUM - cardinal numbers
    PRT - particles or other function words
    X - other: foreign words, typos, abbreviations
    . - punctuation

In [21]:
# Check the word is number or not
def isDigit(word):
    try:
        x = float(word)
        isDigit = True
    except:
        isDigit = False

    return isDigit

In [22]:
import string
# Few morphological rules used to assign unknown word tokens
# Numbers
nums = ['one','two','three','four','five','six','seven','eight','nine','ten']

# Suffixes
noun_suffix = ["ment", "ness", "or", "ry", "scape", "ship", "ty","action", "age", "ance", "cy","an", 
               "dom", "ee", "ence", "er", "hood", "ion", "ism", "ist", "ity"]

verb_suffix = [ "ize","ing","ed","ate", "ify", "ise"]
adjective_suffix = ["ible", "ic", "ish", "ive", "less", "ous","able", "ese", "ful", "i", "ian"]
adverb_suffix = ["ward", "wards", "wise","ly","though","about","above","as","out","further","due","most"]

# Punctuation characters
punct = set(string.punctuation)

### Training for Lexicon and Rule Based Tagging

In [23]:
def getLexiconTag():
    # Rule Based Tagging
    patterns = [
        (r'.*[-a-z]ing$', 'ADJ'),
        (r'.*[-a-z]ed$', 'ADJ'),
        (r'.*[-a-z]$', 'NOUN'),
        (r'.*ing$', 'VERB'),              # gerund
        (r'.*ed$', 'VERB'),               # past tense
        (r'.*es$', 'VERB'),               # 3rd singular present
        (r'.*ould$', 'VERB'),             # modals
        (r'^\'s$','VERB'),
        (r'^\'S$','VERB'),
        (r'^[A-Za-z]\.[A-Za-z]\.$','NOUN'),
        (r'^[A-Za-z]\.$','NOUN'),
        (r'^[0-9]s$', 'NUM'), # cardinal numbers
        (r'.*\'s$', 'NOUN'),              # possessive nouns        
        (r'.*s$', 'NOUN'),               # plural nouns
        (r'^-?[0-9]+(.[0-9]+)?$', 'NUM'), # cardinal numbers
        (r'\*[A-Za-z]\*','X'),             #other: foreign words, typos, abbreviations
        (r'^\*-','X'),
        (r'.*', 'NOUN')                   # nouns
    ]

    # rule based tagger
    rule_based_tagger = nltk.RegexpTagger(patterns)

    # lexicon backed up by the rule-based tagger
    lexicon_tagger = nltk.UnigramTagger(train_set, backoff=rule_based_tagger)
    
    lexicon_tagger = nltk.BigramTagger(train_set, backoff=lexicon_tagger)
    
    return lexicon_tagger

In [24]:
lexicon_tag = getLexiconTag()
lexicon_tag.tag(['*-123'])
lexicon_tag.tag(['exists'])
lexicon_tag.tag(['20s'])

[('20s', 'NOUN')]

In [25]:
def assign_Unknown_Word(word):
    """
    Assign unknown word as per customised tokens
    """
    # Digits - Check if al characters are digit
    if all(char.isdigit() for char in word) or isDigit(word) or (word.lower() in nums):
        return "NUM"

    # Punctuation
    elif all(char in punct for char in word):
        return "."

    # Upper-case
    elif all(char.isupper() for char in word) or word[0].isupper():
        return "NOUN"

    # Nouns
    elif any(word.endswith(suffix) for suffix in noun_suffix):
        return "NOUN"

    # Verbs
    elif any(word.endswith(suffix) for suffix in verb_suffix):
        return "VERB"

    # Adjectives
    elif any(word.endswith(suffix) for suffix in adjective_suffix):
        return "ADJ"

    # Adverbs
    elif any(word.endswith(suffix) for suffix in adverb_suffix):
        return "ADV"
    
    # Adverbs
    elif word in ['that']:
        return "DET"

    return "UNK"

In [26]:
assign_Unknown_Word('private')

'VERB'

In [27]:
# Viterbi Heuristic
def ViterbiForUnknownWords(words, train_bag = train_tagged_words, debug=False):
    #Train as for Lexicon Tag 
    lexicon_tag = getLexiconTag()
    
    
    state = []
    unknown_words=[]
    T = list(set([pair[1] for pair in train_bag]))
    # print(T)
    for key, word in enumerate(words):
        #initialise list of probability column for a given observation
        p = [] 
        for tag in T:
            if key == 0:
                transition_p = tags_df.loc['.', tag]
            else:
                transition_p = tags_df.loc[state[-1], tag]
                
            # compute emission and state probabilities
            emission_out = word_given_tag(words[key], tag)
            emission_p = emission_out[0]/emission_out[1]
            state_probability = emission_p * transition_p 
            p.append(state_probability)
            #if word == 'purchasing':
            #    debug = True
            #    print(emission_out)
            #   print(transition_p)
            #else:
            #    debug = False
        pmax = max(p)
        # getting state for which probability is maximum
        if pmax!=0.0:
            state_max = T[p.index(pmax)]
            if debug:
                print('All Probs:',p)
                print('Max Probs',p.index(pmax))
                print('Predicted Tag:',state_max)
            #print(state_max)
        else:
            if debug:
                print('Customized Method Block')
            # get tagging for unknown words
            state_max = assign_Unknown_Word(word)
            # state_max = lexicon_tag.tag(tokens=[word])[0][1]
            if state_max =='UNK':
                state_max = lexicon_tag.tag(tokens=[word])[0][1]
                if debug:
                    print('Lexicon Block:',state_max)
            
            #List To hold the unknown words and their tag
            unknown_words.append((word,state_max))        
                
        state.append(state_max)
    return list(zip(words, state)),unknown_words

In [28]:
start = time.time()
tagged_seq_new,unknown_words = ViterbiForUnknownWords(test_tagged_words)
end = time.time()
difference = end-start

In [29]:
#word_given_tag('purchasing','VERB')
#lexicon_tag.evaluate(test_set)
# lexicon_tag.tag(tokens=['exists'])
ViterbiForUnknownWords(['Palestinian'],debug=True)
# [pair for pair in train_tagged_words if pair[0] == 'purchasing']

All Probs: [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 7.400381436606114e-06, 0.0, 0.0, 0.0, 0.0]
Max Probs 7
Predicted Tag: ADJ


([('Palestinian', 'ADJ')], [])

#### Evaluating tagging accuracy

#### A - Accuracy Viterbi Along with Lexicon Tagging and customised method to understand nound, proverb, adjective based on language knowledge

In [30]:
print("Time taken in seconds: ", difference)
test_taggings = [tup for sent in test_set for tup in sent]
wronglytagged_words_new =[]
for i in range(len(tagged_seq_new)):
    if test_taggings[i][1] != tagged_seq_new[i][1]:
        wronglytagged_words_new.append((tagged_seq_new[i],test_taggings[i]))
print('Total wrongly tagged words',len(wronglytagged_words_new))
distinct_words_new = list(set([pair[0] for pair in wronglytagged_words_new]))

print('Distinct Word Count',len(distinct_words_new))

# accuracy
check = [i for i, j in zip(tagged_seq_new, test_taggings) if i == j] 
accuracy_new = len(check)/len(tagged_seq_new)
print(accuracy_new)

Time taken in seconds:  48.14597272872925
Total wrongly tagged words 210
Distinct Word Count 180
0.9587183015529782


#### B - Accuracy of only Lexicon and Rule Based Model

In [31]:
getLexiconTag().evaluate(test_set)

0.9530174955769609

In [32]:
# Check Words which are not correctly tagged - Testing
print(list(set([pair[1] for pair in train_tagged_words])))

['VERB', 'PRT', 'ADV', 'NOUN', 'X', 'DET', '.', 'ADJ', 'PRON', 'NUM', 'ADP', 'CONJ']


In [33]:
[w for w in wronglytagged_words_new if w[1][1]=='NOUN']

[(('trade', 'VERB'), ('trade', 'NOUN')),
 (('Private', 'ADJ'), ('Private', 'NOUN')),
 (('slate', 'VERB'), ('slate', 'NOUN')),
 (('homeless', 'ADJ'), ('homeless', 'NOUN')),
 (('test', 'VERB'), ('test', 'NOUN')),
 (('bearing', 'VERB'), ('bearing', 'NOUN')),
 (('work', 'VERB'), ('work', 'NOUN')),
 (('force', 'VERB'), ('force', 'NOUN')),
 (('average', 'ADJ'), ('average', 'NOUN')),
 (('air', 'VERB'), ('air', 'NOUN')),
 (('word-processing', 'VERB'), ('word-processing', 'NOUN')),
 (('declines', 'VERB'), ('declines', 'NOUN')),
 (('covers', 'VERB'), ('covers', 'NOUN')),
 (('pins', 'VERB'), ('pins', 'NOUN')),
 (('estimates', 'VERB'), ('estimates', 'NOUN')),
 (('slowing', 'VERB'), ('slowing', 'NOUN')),
 (('fledgling', 'VERB'), ('fledgling', 'NOUN')),
 (('talk', 'VERB'), ('talk', 'NOUN')),
 (('Palestinian', 'ADJ'), ('Palestinian', 'NOUN')),
 (('half-hour', 'ADJ'), ('half-hour', 'NOUN')),
 (('buy', 'VERB'), ('buy', 'NOUN')),
 (('sell', 'VERB'), ('sell', 'NOUN')),
 (('centers', 'VERB'), ('centers', 

### Compare the tagging accuracies of the modifications with the vanilla Viterbi algorithm

In [34]:
print('Total wrongly tagged words by Vanila Viterbi:',len(wronglytagged_words))
print('Distinct Word Count which are wrongly tagged by Vanila Viterbi:',len(distinct_words))
print('Accuracy by Vanila Viterbi:',accuracy)

Total wrongly tagged words by Vanila Viterbi: 385
Distinct Word Count which are wrongly tagged by Vanila Viterbi: 347
Accuracy by Vanila Viterbi: 0.92431688618046


In [35]:
print('Total wrongly tagged words after correcting Unknown Words:',len(wronglytagged_words_new))
print('Distinct Word Count after correcting Unknown Words:',len(distinct_words_new))
print('Accuracy by Vanila Viterbi:',accuracy_new)

Total wrongly tagged words after correcting Unknown Words: 210
Distinct Word Count after correcting Unknown Words: 180
Accuracy by Vanila Viterbi: 0.9587183015529782


In [36]:
print('Total Unknown Words:',len(unknown_words))

Total Unknown Words: 307


### List down cases which were incorrectly tagged by original POS tagger and got corrected by your modifications

In [37]:
correctly_tagged_unknown_words=[]
for unk in unknown_words:
    for tst in wronglytagged_words:
        if unk[0] == tst[1][0] and tst[1][1]==unk[1]:
            correctly_tagged_unknown_words.append(unk)    

In [38]:
print('Out of {0} number of unknown words, modfified Viterbi has correctly POS tagged {1} numbers of words'
      .format(len(wronglytagged_words),len(correctly_tagged_unknown_words)))

Out of 385 number of unknown words, modfified Viterbi has correctly POS tagged 207 numbers of words


In [39]:
print('Correctly Tagged Unknownd Words are:\n', list([i[0] for i in correctly_tagged_unknown_words ]))

Correctly Tagged Unknownd Words are:
 ['Overseas', 'Unemployment', 'paycheck', 'reasonably', 'Tokio', 'protocols', 'preventative', '20.5', 'acquirers', '154.2', 'chalk', 'schoolchildren', 'Catch-22', 'emigres', '*T*-133', '1953', '1955', 'addiction', '1,200', 'Mogavero', 'Piscataway', '*T*-253', 'reds', 'Rhone', '133.7', '94', 'octogenarians', '*T*-222', 'belfries', 'Anglia', '405', '*T*-102', 'Communication', 'importer', '*T*-128', '*T*-129', 'mail', 'alternatively', 'intrusions', 'Sloan', '43.875', 'reluctance', 'lobbies', '133.8', '2.47', '7.4', '2.30', 'modification', '*-130', '13.625', 'blindfold', 'Anything', 'Northampton', 'protein', 'Fundamentalists', 'bags', 'workplace', 'Legend', 'Iran-Contra', 'Heidelberg', 'approaches', 'college-bowl', 'competitions', 'coffee', 'Velcro', 'platinum', 'platinum', 'Johnson-era', 'noncompetitively', 'Wilfred', 'Corrigan', 'derivatives', 'binders', '93.9', '1.19', '92.9', '1.18', 'Demand', 'Berson', 'longevity', 'rarely', 'deplorable', 'insurgen

In [40]:
incorrectly_tagged_unknown_words=[]
for corr in unknown_words:
    for unk in wronglytagged_words:
        if unk[1][0]==corr[0] and unk[1][1]!=corr[1]:
            incorrectly_tagged_unknown_words.append(unk[1])  

In [41]:
print('Out of {0} number of unknown words, modfified Viterbi has unable to correctly POS tagged {1} numbers of words.'
      .format(len(unknown_words),len(incorrectly_tagged_unknown_words)))

print('Incorrectly Tagged Unknownd Words are:\n', list([i[0] for i in incorrectly_tagged_unknown_words ]))

Out of 307 number of unknown words, modfified Viterbi has unable to correctly POS tagged 58 numbers of words.
Incorrectly Tagged Unknownd Words are:
 ['pre-1917', 'Though', 'slate', 'cross-border', 'safe', 'pre-existing', 'American-style', 'sometimes-exhausting', 'manmade-fiber', 'low-cost', 'supplemental', 'anti-drug', 'computer-generated', 'certified', 'void', 'Muzzling', 'side-crash', 'word-processing', 'multinational', 'wheel-loader', 'desultory', 'herself', 'non-core', 'twice', 'apparent', 'razor-thin', 'fledgling', 'triple', 'Washington-based', 'high-rise', 'pre-cooked', 'entertaining', 'confrontational', 'shareholder-rights', 'unwanted', 'low-tech', 'cumbersome', 'day-care', 'monopoly', '20s', '30s', 'computer-system-design', 'more-advanced', '100-megabyte', 'floral', '70-a-share', 'price-support', 'direct-investment', 'avid', 'cable', 'seven-million-ton', 'do-it-yourself', 'raring', 'lap-shoulder', 'rear', 'malignant', 'war-rationed', 'improper']


### Reasons for Incorrect Tagging:
- There are few words for which the emission probability is much hgher in Train set than test set which is actually 
influencing the prediction on test set.

#### Testing

In [42]:
## Testing
from nltk.tokenize import word_tokenize,sent_tokenize
sentence_test = "Android is a mobile operating system developed by Google. Android has been the best-selling OS worldwide on smartphones since 2011 and on tablets since 2013. Google and Twitter made a deal in 2015 that gave Google access to Twitter's firehose. Twitter is an online news and social networking service on which users post and interact with messages known as tweets. Before entering politics, Donald Trump was a domineering businessman and a television personality. The 2018 FIFA World Cup is the 21st FIFA World Cup, an international football tournament contested once every four years. This is the first World Cup to be held in Eastern Europe and the 11th time that it has been held in Europe. Show me the cheapest round trips from Dallas to Atlanta.I would like to see flights from Denver to Philadelphia. Show me the price of the flights leaving Atlanta at about 3 in the afternoon and arriving in San Francisco. NASA invited social media users to experience the launch of ICESAT-2 Satellite."
words = word_tokenize(sentence_test)

start = time.time()
tagged_seq = Viterbi(words)
end = time.time()
difference = end-start

In [43]:
print(tagged_seq)
print(difference)

[('Android', 'VERB'), ('is', 'VERB'), ('a', 'DET'), ('mobile', 'ADJ'), ('operating', 'NOUN'), ('system', 'NOUN'), ('developed', 'VERB'), ('by', 'ADP'), ('Google', 'VERB'), ('.', '.'), ('Android', 'VERB'), ('has', 'VERB'), ('been', 'VERB'), ('the', 'DET'), ('best-selling', 'ADJ'), ('OS', 'VERB'), ('worldwide', 'VERB'), ('on', 'ADP'), ('smartphones', 'VERB'), ('since', 'ADP'), ('2011', 'VERB'), ('and', 'CONJ'), ('on', 'ADP'), ('tablets', 'NOUN'), ('since', 'ADP'), ('2013', 'VERB'), ('.', '.'), ('Google', 'VERB'), ('and', 'CONJ'), ('Twitter', 'VERB'), ('made', 'VERB'), ('a', 'DET'), ('deal', 'NOUN'), ('in', 'ADP'), ('2015', 'VERB'), ('that', 'ADP'), ('gave', 'VERB'), ('Google', 'VERB'), ('access', 'NOUN'), ('to', 'PRT'), ('Twitter', 'VERB'), ("'s", 'PRT'), ('firehose', 'VERB'), ('.', '.'), ('Twitter', 'VERB'), ('is', 'VERB'), ('an', 'DET'), ('online', 'VERB'), ('news', 'NOUN'), ('and', 'CONJ'), ('social', 'ADJ'), ('networking', 'NOUN'), ('service', 'NOUN'), ('on', 'ADP'), ('which', 'DET')

In [44]:
start = time.time()
tagged_seq = ViterbiForUnknownWords(words)
end = time.time()
difference = end-start

In [45]:
print(tagged_seq)
print(difference)

([('Android', 'NOUN'), ('is', 'VERB'), ('a', 'DET'), ('mobile', 'ADJ'), ('operating', 'NOUN'), ('system', 'NOUN'), ('developed', 'VERB'), ('by', 'ADP'), ('Google', 'NOUN'), ('.', '.'), ('Android', 'NOUN'), ('has', 'VERB'), ('been', 'VERB'), ('the', 'DET'), ('best-selling', 'ADJ'), ('OS', 'NOUN'), ('worldwide', 'NOUN'), ('on', 'ADP'), ('smartphones', 'NOUN'), ('since', 'ADP'), ('2011', 'NUM'), ('and', 'CONJ'), ('on', 'ADP'), ('tablets', 'NOUN'), ('since', 'ADP'), ('2013', 'NUM'), ('.', '.'), ('Google', 'NOUN'), ('and', 'CONJ'), ('Twitter', 'NOUN'), ('made', 'VERB'), ('a', 'DET'), ('deal', 'NOUN'), ('in', 'ADP'), ('2015', 'NUM'), ('that', 'ADP'), ('gave', 'VERB'), ('Google', 'NOUN'), ('access', 'NOUN'), ('to', 'PRT'), ('Twitter', 'NOUN'), ("'s", 'PRT'), ('firehose', 'NOUN'), ('.', '.'), ('Twitter', 'NOUN'), ('is', 'VERB'), ('an', 'DET'), ('online', 'NOUN'), ('news', 'NOUN'), ('and', 'CONJ'), ('social', 'ADJ'), ('networking', 'NOUN'), ('service', 'NOUN'), ('on', 'ADP'), ('which', 'DET'), 

### Conclusion
-  Modified Viterbi is performing well e.g. it is correctly Identifing the nouns : Android, smartphones and other tags as well.
